 # 🔍 DataFlip MX - Descubrimiento Automático de Nichos



 **Objetivo:** Explorar TODO Mercado Libre para encontrar oportunidades ocultas



 **Estrategia:**

 1. Analizar categorías completas

 2. Identificar productos con alta demanda y baja competencia

 3. Detectar tendencias emergentes

 4. Generar lista de nichos prometedores automáticamente

In [9]:
# === IMPORTAR LIBRERÍAS ===
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import time
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")


✅ Librerías importadas correctamente


In [30]:
# === CONFIGURACIÓN CON HEADERS MEJORADOS ===
BASE_URL = "https://api.mercadolibre.com"
SITE_ID = "MLM"  # México
REQUEST_DELAY = 2  # Aumentado para evitar rate limiting

# Headers que simulan un navegador real
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json',
    'Accept-Language': 'es-MX,es;q=0.9,en;q=0.8',
    'Referer': 'https://www.mercadolibre.com.mx/',
    'Origin': 'https://www.mercadolibre.com.mx'
}

# Parámetros de búsqueda
MIN_SOLD_QUANTITY = 50  # Mínimo de ventas para considerar el producto
MAX_LISTINGS = 30       # Máximo de listings (baja competencia)
MIN_PRICE = 200         # Precio mínimo (evitar productos muy baratos)
MAX_PRICE = 5000        # Precio máximo (enfoque en ticket medio)

print("🎯 Parámetros de búsqueda configurados")


🎯 Parámetros de búsqueda configurados


In [31]:
# === FUNCIONES DE EXPLORACIÓN MEJORADAS ===

def make_request(url: str, params: dict = None, max_retries: int = 3) -> dict:
    """
    Hace un request con reintentos y manejo de errores mejorado
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(
                url, 
                params=params, 
                headers=HEADERS,
                timeout=15
            )
            
            # Si recibimos 403, intentar sin algunos headers
            if response.status_code == 403 and attempt < max_retries - 1:
                print(f"⚠️  403 recibido, reintentando sin Origin/Referer...")
                time.sleep(REQUEST_DELAY * 2)
                
                # Intentar con headers más simples
                simple_headers = {
                    'User-Agent': HEADERS['User-Agent'],
                    'Accept': 'application/json'
                }
                response = requests.get(url, params=params, headers=simple_headers, timeout=15)
            
            response.raise_for_status()
            return response.json()
        
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 403:
                print(f"❌ Error 403 (Forbidden) - Intento {attempt + 1}/{max_retries}")
                if attempt < max_retries - 1:
                    wait_time = REQUEST_DELAY * (attempt + 2)
                    print(f"   Esperando {wait_time}s antes de reintentar...")
                    time.sleep(wait_time)
                else:
                    print("\n💡 SOLUCIÓN: La API de MercadoLibre está bloqueando el acceso.")
                    print("   Opciones:")
                    print("   1. Usar un VPN")
                    print("   2. Esperar unos minutos y reintentar")
                    print("   3. Usar el approach alternativo con búsquedas directas (ver más abajo)")
                    return {}
            else:
                print(f"❌ Error HTTP {e.response.status_code}: {e}")
                return {}
        
        except Exception as e:
            print(f"❌ Error: {e}")
            if attempt < max_retries - 1:
                time.sleep(REQUEST_DELAY)
            else:
                return {}
    
    return {}

def get_all_categories(site_id: str = SITE_ID) -> pd.DataFrame:
    """
    Obtiene todas las categorías y subcategorías de Mercado Libre
    """
    url = f"{BASE_URL}/sites/{site_id}/categories"
    
    data = make_request(url)
    
    if data:
        df = pd.DataFrame(data)
        print(f"✅ {len(df)} categorías principales encontradas")
        return df
    else:
        print("⚠️  No se pudieron obtener categorías. Usando approach alternativo...")
        return pd.DataFrame()

def search_products(query: str, limit: int = 50, sort: str = 'relevance') -> List[Dict]:
    """
    Busca productos por término de búsqueda (approach alternativo)
    """
    url = f"{BASE_URL}/sites/{SITE_ID}/search"
    params = {
        'q': query,
        'limit': limit,
        'sort': sort
    }
    
    data = make_request(url, params)
    
    if data and 'results' in data:
        print(f"✅ {len(data['results'])} productos encontrados para '{query}'")
        return data['results']
    return []

def search_in_category(category_id: str, sort: str = 'sold_quantity_desc', limit: int = 50) -> List[Dict]:
    """
    Busca productos en una categoría específica
    """
    url = f"{BASE_URL}/sites/{SITE_ID}/search"
    params = {
        'category': category_id,
        'limit': limit,
        'sort': sort,
        'offset': 0
    }
    
    data = make_request(url, params)
    
    if data and 'results' in data:
        return data['results']
    return []

def parse_product(item: Dict) -> Dict:
    """
    Extrae información relevante de un producto
    """
    return {
        'id': item.get('id'),
        'titulo': item.get('title'),
        'precio': item.get('price'),
        'vendidos': item.get('sold_quantity', 0),
        'condicion': item.get('condition'),
        'envio_gratis': item.get('shipping', {}).get('free_shipping', False),
        'categoria_id': item.get('category_id'),
        'vendedor_id': item.get('seller', {}).get('id'),
        'link': item.get('permalink'),
    }

In [32]:
# === PASO 1: EXPLORAR CATEGORÍAS PRINCIPALES ===
print("\n" + "="*60)
print("🗂️ PASO 1: MAPEO DE CATEGORÍAS")
print("="*60 + "\n")

df_categories = get_all_categories()

if not df_categories.empty:
    print("\n📊 Categorías principales de Mercado Libre MX:\n")
    print(df_categories[['id', 'name']].to_string(index=False))



🗂️ PASO 1: MAPEO DE CATEGORÍAS

⚠️  403 recibido, reintentando sin Origin/Referer...
❌ Error 403 (Forbidden) - Intento 1/3
   Esperando 4s antes de reintentar...
⚠️  403 recibido, reintentando sin Origin/Referer...
❌ Error 403 (Forbidden) - Intento 2/3
   Esperando 6s antes de reintentar...
❌ Error 403 (Forbidden) - Intento 3/3

💡 SOLUCIÓN: La API de MercadoLibre está bloqueando el acceso.
   Opciones:
   1. Usar un VPN
   2. Esperar unos minutos y reintentar
   3. Usar el approach alternativo con búsquedas directas (ver más abajo)
⚠️  No se pudieron obtener categorías. Usando approach alternativo...


In [21]:
# === CATEGORÍAS PRIORIZADAS PARA REVENTA ===
print("\n" + "="*60)
print("🎯 CATEGORÍAS PRIORITARIAS")
print("="*60 + "\n")

# Categorías con alto potencial de reventa
PRIORITY_CATEGORIES = {
    'MLM1000': 'Electrónica, Audio y Video',
    'MLM1144': 'Consolas y Videojuegos',
    'MLM1168': 'Computación',
    'MLM1182': 'Cámaras y Accesorios',
    'MLM1276': 'Deportes y Fitness',
    'MLM3937': 'Libros, Revistas y Comics',
    'MLM1039': 'Instrumentos Musicales',
    'MLM1648': 'Herramientas',
    'MLM1499': 'Industrias y Oficinas',
    'MLM1403': 'Juegos y Juguetes',
}

print("Analizaremos estas categorías:\n")
for cat_id, cat_name in PRIORITY_CATEGORIES.items():
    print(f"   📁 {cat_name} ({cat_id})")

print(f"\n⏳ Esto tomará aprox. {len(PRIORITY_CATEGORIES) * 2} minutos...")



🎯 CATEGORÍAS PRIORITARIAS

Analizaremos estas categorías:

   📁 Electrónica, Audio y Video (MLM1000)
   📁 Consolas y Videojuegos (MLM1144)
   📁 Computación (MLM1168)
   📁 Cámaras y Accesorios (MLM1182)
   📁 Deportes y Fitness (MLM1276)
   📁 Libros, Revistas y Comics (MLM3937)
   📁 Instrumentos Musicales (MLM1039)
   📁 Herramientas (MLM1648)
   📁 Industrias y Oficinas (MLM1499)
   📁 Juegos y Juguetes (MLM1403)

⏳ Esto tomará aprox. 20 minutos...


In [ ]:
# === PASO 2: EXPLORACIÓN MASIVA ===
print("\n" + "="*60)
print("🔍 PASO 2: EXPLORACIÓN MASIVA DE PRODUCTOS")
print("="*60 + "\n")

all_products = []
category_stats = []

for cat_id, cat_name in PRIORITY_CATEGORIES.items():
    print(f"\n📊 Analizando: {cat_name}")
    
    # Buscar top vendidos
    products = search_in_category(cat_id, sort='sold_quantity_desc', limit=50)
    
    if products:
        parsed = [parse_product(p) for p in products]
        df_cat = pd.DataFrame(parsed)
        df_cat['categoria_nombre'] = cat_name
        
        all_products.append(df_cat)
        
        # Estadísticas de la categoría
        stats = {
            'categoria': cat_name,
            'num_productos': len(df_cat),
            'precio_promedio': df_cat['precio'].mean(),
            'ventas_totales': df_cat['vendidos'].sum(),
            'ventas_promedio': df_cat['vendidos'].mean(),
        }
        category_stats.append(stats)
        
        print(f"   ✅ {len(df_cat)} productos | Ventas totales: {stats['ventas_totales']}")
    
    else:
        print(f"   ⚠️  Sin resultados")
    
    time.sleep(REQUEST_DELAY)

# Combinar todos los productos
if all_products:
    df_all = pd.concat(all_products, ignore_index=True)
    print(f"\n✅ Total de productos analizados: {len(df_all)}")
else:
    print("\n❌ No se obtuvieron productos")
    df_all = pd.DataFrame()


In [ ]:
# === PASO 3: IDENTIFICAR NICHOS ESPECÍFICOS ===
if not df_all.empty:
    print("\n" + "="*60)
    print("🎯 PASO 3: IDENTIFICACIÓN DE NICHOS")
    print("="*60 + "\n")
    
    # Estrategia: Agrupar productos similares y analizar competencia
    
    # 1. Extraer palabras clave de títulos (primeras 3 palabras significativas)
    def extract_keywords(title: str) -> str:
        """Extrae las primeras 3-4 palabras clave del título"""
        # Remover palabras comunes
        stopwords = {'con', 'para', 'sin', 'de', 'en', 'y', 'o', 'a', 'nuevo', 'usado'}
        words = title.lower().split()[:6]
        keywords = [w for w in words if w not in stopwords and len(w) > 2]
        return ' '.join(keywords[:3])
    
    df_all['keywords'] = df_all['titulo'].apply(extract_keywords)
    
    # 2. Agrupar por keywords y analizar
    niche_analysis = df_all.groupby('keywords').agg({
        'id': 'count',              # Número de listings (competencia)
        'vendidos': ['sum', 'mean', 'max'],
        'precio': ['mean', 'std'],
        'envio_gratis': 'mean'
    }).round(2)
    
    niche_analysis.columns = ['_'.join(col).strip() for col in niche_analysis.columns.values]
    niche_analysis = niche_analysis.rename(columns={
        'id_count': 'num_listings',
        'vendidos_sum': 'ventas_totales',
        'vendidos_mean': 'ventas_promedio',
        'vendidos_max': 'ventas_max',
        'precio_mean': 'precio_promedio',
        'precio_std': 'precio_varianza',
        'envio_gratis_mean': 'pct_envio_gratis'
    })
    
    # 3. Calcular score de oportunidad
    niche_analysis['ratio_demanda_competencia'] = (
        niche_analysis['ventas_totales'] / niche_analysis['num_listings']
    )
    
    # 4. Filtrar nichos prometedores
    nichos_prometedores = niche_analysis[
        (niche_analysis['ventas_totales'] >= MIN_SOLD_QUANTITY) &
        (niche_analysis['num_listings'] <= MAX_LISTINGS) &
        (niche_analysis['precio_promedio'] >= MIN_PRICE) &
        (niche_analysis['precio_promedio'] <= MAX_PRICE)
    ].sort_values('ratio_demanda_competencia', ascending=False)
    
    print(f"🎉 {len(nichos_prometedores)} nichos prometedores encontrados\n")
    print("🏆 TOP 20 NICHOS CON MEJOR RATIO DEMANDA/COMPETENCIA:\n")
    print(nichos_prometedores.head(20))


In [ ]:
# === VISUALIZACIÓN: TOP NICHOS ===
if not df_all.empty and len(nichos_prometedores) > 0:
    top_nichos = nichos_prometedores.head(15)
    
    fig = go.Figure()
    
    # Scatter: Competencia vs Demanda
    fig.add_trace(go.Scatter(
        x=top_nichos['num_listings'],
        y=top_nichos['ventas_totales'],
        mode='markers+text',
        marker=dict(
            size=top_nichos['precio_promedio'] / 50,
            color=top_nichos['ratio_demanda_competencia'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Ratio<br>Demanda/Comp")
        ),
        text=top_nichos.index,
        textposition='top center',
        hovertemplate=(
            '<b>%{text}</b><br>' +
            'Listings: %{x}<br>' +
            'Ventas: %{y}<br>' +
            'Precio: $%{marker.size:.0f}<br>' +
            '<extra></extra>'
        )
    ))
    
    fig.update_layout(
        title='🎯 Mapa de Oportunidades: Demanda vs Competencia',
        xaxis_title='Número de Listings (Competencia)',
        yaxis_title='Ventas Totales (Demanda)',
        height=600,
        template='plotly_white',
        showlegend=False
    )
    
    # Zona ideal (baja competencia, alta demanda)
    fig.add_shape(
        type="rect",
        x0=0, y0=MIN_SOLD_QUANTITY,
        x1=MAX_LISTINGS, y1=nichos_prometedores['ventas_totales'].max(),
        fillcolor="green",
        opacity=0.1,
        line_width=0,
    )
    
    fig.add_annotation(
        x=MAX_LISTINGS/2, y=nichos_prometedores['ventas_totales'].max() * 0.9,
        text="ZONA IDEAL",
        showarrow=False,
        font=dict(size=14, color="green")
    )
    
    fig.show()


In [ ]:
# === ANÁLISIS POR CATEGORÍA ===
if category_stats:
    df_cat_stats = pd.DataFrame(category_stats)
    df_cat_stats = df_cat_stats.sort_values('ventas_totales', ascending=False)
    
    print("\n" + "="*60)
    print("📊 ESTADÍSTICAS POR CATEGORÍA")
    print("="*60 + "\n")
    print(df_cat_stats.round(2))
    
    # Visualizar
    fig = px.bar(
        df_cat_stats.head(10),
        x='categoria',
        y='ventas_totales',
        color='precio_promedio',
        title='📈 Categorías con Mayor Volumen de Ventas',
        labels={'ventas_totales': 'Ventas Totales', 'categoria': 'Categoría'},
        height=500,
        color_continuous_scale='Blues'
    )
    
    fig.update_layout(
        xaxis_tickangle=-45,
        template='plotly_white'
    )
    
    fig.show()


In [ ]:
# === DETECCIÓN DE PRODUCTOS "GANADORES" ===
if not df_all.empty:
    print("\n" + "="*60)
    print("🏆 PRODUCTOS INDIVIDUALES MÁS PROMETEDORES")
    print("="*60 + "\n")
    
    # Productos con muchas ventas pero precio accesible
    ganadores = df_all[
        (df_all['vendidos'] > 100) &
        (df_all['precio'] >= MIN_PRICE) &
        (df_all['precio'] <= MAX_PRICE)
    ].sort_values('vendidos', ascending=False)
    
    print(f"🎯 {len(ganadores)} productos ganadores identificados\n")
    print("Top 20:\n")
    print(ganadores[['titulo', 'precio', 'vendidos', 'categoria_nombre']].head(20).to_string(index=False))
    
    # Visualizar
    if len(ganadores) > 0:
        top_winners = ganadores.head(15)
        
        fig = px.scatter(
            top_winners,
            x='precio',
            y='vendidos',
            size='vendidos',
            color='categoria_nombre',
            hover_name='titulo',
            title='🏆 Productos Ganadores: Precio vs Ventas',
            labels={'precio': 'Precio (MXN)', 'vendidos': 'Cantidad Vendida'},
            height=600
        )
        
        fig.update_layout(template='plotly_white')
        fig.show()


In [ ]:
# === GENERAR LISTA FINAL DE NICHOS RECOMENDADOS ===
if not df_all.empty and len(nichos_prometedores) > 0:
    print("\n" + "="*60)
    print("✅ LISTA FINAL: NICHOS PARA INVESTIGAR")
    print("="*60 + "\n")
    
    # Top 15 nichos con contexto adicional
    top_15_nichos = nichos_prometedores.head(15).copy()
    
    # Agregar recomendación
    def get_recommendation(row):
        ratio = row['ratio_demanda_competencia']
        ventas = row['ventas_totales']
        
        if ratio > 20 and ventas > 200:
            return "🔥 ATACAR YA"
        elif ratio > 10 and ventas > 100:
            return "✅ VALIDAR"
        else:
            return "⚠️ INVESTIGAR"
    
    top_15_nichos['recomendacion'] = top_15_nichos.apply(get_recommendation, axis=1)
    
    print("🎯 NICHOS PRIORIZADOS:\n")
    display_cols = [
        'ventas_totales', 'num_listings', 'precio_promedio', 
        'ratio_demanda_competencia', 'recomendacion'
    ]
    print(top_15_nichos[display_cols])
    
    # Crear lista de nichos para usar en otros notebooks
    nichos_lista = top_15_nichos.index.tolist()
    
    print(f"\n📋 NICHOS A ANALIZAR EN PROFUNDIDAD ({len(nichos_lista)}):")
    for i, nicho in enumerate(nichos_lista, 1):
        print(f"   {i}. {nicho}")


In [ ]:
# === EXPORTAR RESULTADOS ===
if not df_all.empty:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Exportar dataset completo
    df_all.to_csv(f'data/raw/discovery_productos_{timestamp}.csv', index=False, encoding='utf-8-sig')
    
    # Exportar nichos prometedores
    if len(nichos_prometedores) > 0:
        nichos_prometedores.to_csv(f'data/analytics/nichos_descubiertos_{timestamp}.csv', encoding='utf-8-sig')
    
    # Exportar stats de categorías
    if category_stats:
        df_cat_stats.to_csv(f'data/analytics/categoria_stats_{timestamp}.csv', index=False, encoding='utf-8-sig')
    
    print(f"\n✅ Datos exportados con timestamp: {timestamp}")
    
    print("\n" + "="*60)
    print("🎉 DESCUBRIMIENTO COMPLETADO")
    print("="*60)
    print(f"\n📊 Resumen:")
    print(f"   - {len(df_all)} productos analizados")
    print(f"   - {len(nichos_prometedores)} nichos prometedores")
    print(f"   - {len(PRIORITY_CATEGORIES)} categorías exploradas")
    
    print("\n📋 Próximos pasos:")
    print("   1. Revisar la lista de nichos generada")
    print("   2. Ejecutar 01_mercadolibre_api.ipynb con estos nichos")
    print("   3. Validar demanda con Google Trends")
    print("   4. Calcular márgenes esperados")